In [23]:
import collections
import random
import simpy


In [24]:
RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SIM_TIME = 120  # Simulate until


In [25]:
def moviegoer(env, movie, num_tickets, theater):
    """A moviegoer tries to by a number of tickets (*num_tickets*) for
    a certain *movie* in a *theater*.

    If the movie becomes sold out, she leaves the theater. If she gets
    to the counter, she tries to buy a number of tickets. If not enough
    tickets are left, she argues with the teller and leaves.

    If at most one ticket is left after the moviegoer bought her
    tickets, the *sold out* event for this movie is triggered causing
    all remaining moviegoers to leave.

    """
    with theater.counter.request() as my_turn:
        # Wait until its our turn or until the movie is sold out
        result = yield my_turn | theater.sold_out[movie]

        # Check if it's our turn or if movie is sold out
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            return

        # Check if enough tickets left.
        if theater.available[movie] < num_tickets:
            # Moviegoer leaves after some discussion
            yield env.timeout(random.expovariate(1))
            return

        # Buy tickets
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Trigger the "sold out" event for the movie
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(random.expovariate(1))

In [26]:
def customer_arrivals(env, theater):
    """Create new *moviegoers* until the sim time reaches 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 8)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))

In [27]:
Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')


In [28]:
# Setup and start the simulation
print('Moviegoer')
random.seed(RANDOM_SEED)
env = simpy.Environment()

Moviegoer


In [29]:
# Create movie theater
counter = simpy.Resource(env, capacity=1)
movies = ['Toy Story', 'Mary Poppins', 'The Wizard of Oz', 'E.T.','Home Alone', 'Finding Nemo','Cars','Moana','Minions']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)


In [30]:
# Start process and run
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

In [31]:
# Analysis/results
for movie in movies:
    if theater.sold_out[movie]:
        print('Movie "%s" sold out %.1f minutes after ticket counter '
              'opening.' % (movie, theater.when_sold_out[movie]))
        print('  Number of people leaving queue when film sold out: %s' %
              theater.num_renegers[movie])


Movie "Toy Story" sold out 91.5 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 6
Movie "Mary Poppins" sold out 109.4 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 2
Movie "The Wizard of Oz" sold out 102.1 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 4
Movie "E.T." sold out 92.5 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 19
Movie "Home Alone" sold out 105.9 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 9
Movie "Finding Nemo" sold out 115.7 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 3
Movie "Cars" sold out 107.0 minutes after ticket counter opening.
  Number of people leaving queue when film sold out: 5
Movie "Moana" sold out 105.3 minutes after ticket counter opening.
  Number of people leaving queue when film sold 